In [1]:
import torch
print(f"Is GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Using device: {torch.cuda.get_device_name(0)}")

Is GPU available: True
Using device: Tesla T4


In [2]:
!pip install transformers torch scikit-learn tensorflow


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from transformers import pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch

In [4]:
# Ensure GPU is used for Zero-Shot Classifier
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
# Load the dataset
file_path = "/content/AMMUSED (1).csv"
dataset = pd.read_csv(file_path)

In [6]:
# Focus on relevant columns
comments = dataset['comment'].astype(str)
labels = dataset['label_y']

In [7]:
# Define possible stance labels
possible_labels = ["agree", "disagree", "comment", "query"]

In [8]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [9]:
# Preprocess the text data
max_sequence_length = 100
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [10]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, encoded_labels, test_size=0.2, random_state=42
)

In [11]:
# Zero-Shot Classification Predictions
def get_zero_shot_predictions(comments, possible_labels):
    predictions = []
    for i, comment in enumerate(comments):
        # Show progress
        if i % 100 == 0:
            print(f"Processing {i}/{len(comments)} comments...")
        result = zero_shot_classifier(comment, candidate_labels=possible_labels)
        predicted_label = result['labels'][0]  # Take the label with the highest score
        predictions.append(predicted_label)
    return predictions

In [12]:
# Apply zero-shot classification on the dataset
predicted_labels = get_zero_shot_predictions(comments, possible_labels)

Processing 0/31779 comments...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing 100/31779 comments...
Processing 200/31779 comments...
Processing 300/31779 comments...
Processing 400/31779 comments...
Processing 500/31779 comments...
Processing 600/31779 comments...
Processing 700/31779 comments...
Processing 800/31779 comments...
Processing 900/31779 comments...
Processing 1000/31779 comments...
Processing 1100/31779 comments...
Processing 1200/31779 comments...
Processing 1300/31779 comments...
Processing 1400/31779 comments...
Processing 1500/31779 comments...
Processing 1600/31779 comments...
Processing 1700/31779 comments...
Processing 1800/31779 comments...
Processing 1900/31779 comments...
Processing 2000/31779 comments...
Processing 2100/31779 comments...
Processing 2200/31779 comments...
Processing 2300/31779 comments...
Processing 2400/31779 comments...
Processing 2500/31779 comments...
Processing 2600/31779 comments...
Processing 2700/31779 comments...
Processing 2800/31779 comments...
Processing 2900/31779 comments...
Processing 3000/31779 c

In [13]:
# Map predicted labels back to encoded labels
predicted_encoded_labels = label_encoder.transform(predicted_labels)

In [14]:
# Evaluate the model
accuracy = accuracy_score(encoded_labels, predicted_encoded_labels)
print(f"Zero-Shot Stance Detection Accuracy: {accuracy:.4f}")

Zero-Shot Stance Detection Accuracy: 0.3588


In [15]:
# Save the predictions and their associated comments
predictions_df = pd.DataFrame({
    'comment': comments,
    'true_label': labels,
    'predicted_label': predicted_labels
})

In [16]:
# Save predictions to a CSV file
predictions_df.to_csv("zero_shot_predictions.csv", index=False)
print("Predictions saved to zero_shot_predictions.csv")

Predictions saved to zero_shot_predictions.csv
